In [32]:
from fipy import *
import mayavi
import math
import numpy as np
from fipy.tools import dump

In [35]:
mesh = Gmsh2DIn3DSpace('''
...     radius = 15.0;
...     cellSize = 0.4;
...     
...     // create inner 1/8 shell
...     Point(1) = {0, 0, 0, cellSize};
...     Point(2) = {-radius, 0, 0, cellSize};
...     Point(3) = {0, radius, 0, cellSize};
...     Point(4) = {0, 0, radius, cellSize};
...     Circle(1) = {2, 1, 3};
...     Circle(2) = {4, 1, 2};
...     Circle(3) = {4, 1, 3};
...     Line Loop(1) = {1, -3, 2} ;
...     Ruled Surface(1) = {1};
...     
...     // create remaining 7/8 inner shells
...     t1[] = Rotate {{0,0,1},{0,0,0},Pi/2} {Duplicata{Surface{1};}};
...     t2[] = Rotate {{0,0,1},{0,0,0},Pi} {Duplicata{Surface{1};}};
...     t3[] = Rotate {{0,0,1},{0,0,0},Pi*3/2} {Duplicata{Surface{1};}};
...     t4[] = Rotate {{0,1,0},{0,0,0},-Pi/2} {Duplicata{Surface{1};}};
...     t5[] = Rotate {{0,0,1},{0,0,0},Pi/2} {Duplicata{Surface{t4[0]};}};
...     t6[] = Rotate {{0,0,1},{0,0,0},Pi} {Duplicata{Surface{t4[0]};}};
...     t7[] = Rotate {{0,0,1},{0,0,0},Pi*3/2} {Duplicata{Surface{t4[0]};}};
...     
...     // create entire inner and outer shell
...     Surface Loop(100)={1,t1[0],t2[0],t3[0],t7[0],t4[0],t5[0],t6[0]};
... ''', order=2).extrude(extrudeFunc=lambda r: 1.1 * r) 

#gmsh code for creating meshed sphere is given above
#set up variables, parameters, and initial condition
phi = CellVariable(name=r"$\phi$", mesh=mesh) 
phi.setValue(GaussianNoiseVariable(mesh=mesh,mean=0,variance=0.04)) 
PHI = phi.arithmeticFaceValue 
a = epsilon = 1.
qn =  1.5
D = 1.
K = 1.
u4 = 120.
u3 = -40.
tau = 20.
dexp = -7
elapsed = 0

#define the conserved dynamics equation
sourcey = (u4*0.5*PHI*PHI+u3*PHI+tau)+D*K*qn**4
eq = (TransientTerm()== DiffusionTerm(coeff=sourcey)+DiffusionTerm(coeff=(2*qn**2,D*K))+DiffusionTerm(coeff=(1., 1., D*K))) 


In [36]:
# set the total integration time "duration" and evolve the dynamics
duration = 2
while elapsed < duration:
    dt = min(0.005, math.exp(dexp))
    elapsed += dt
    dexp += 0.005
    eq.solve(phi, dt=dt) 

In [37]:
# view the result
view = Viewer(vars=phi)

In [39]:
# extrapolate the solution onto points on sphere


x3d,y3d,z3d=mesh.faceCenters
valc = np.array(phi.arithmeticFaceValue)

rr = -1.
r3d=(x3d**2+y3d**2)**0.5
theta3d=np.zeros(len(z3d))
pp3d=np.zeros(len(z3d))
x3dn = np.zeros(len(z3d))
y3dn = np.zeros(len(z3d))
z3dn = np.zeros(len(z3d))


for i in range(0,len(z3d)):
    pp3d[i]=np.arctan2(y3d[i],x3d[i])
    rad = (x3d[i]**2+y3d[i]**2+z3d[i]**2)**0.5
    theta3d[i]=np.arccos(z3d[i]/rad)
    x3dn[i]=(rad+rr*valc[i])*np.cos(pp3d[i])*np.sin(theta3d[i])
    y3dn[i]=(rad+rr*valc[i])*np.sin(pp3d[i])*np.sin(theta3d[i])
    z3dn[i]=(rad+rr*valc[i])*np.cos(theta3d[i])

valc=(valc-valc.min())/(valc.max()-valc.min())

In [40]:
# plot the points as a surface

from mayavi import mlab
mlab.figure(fgcolor=(0., 0., 0.), bgcolor=(0, 0, 0))

pts = mlab.points3d(x3dn,y3dn,z3dn,scale_factor=0.5,colormap='YlOrBr')
pts.glyph.scale_mode = 'scale_by_vector'
pts.mlab_source.dataset.point_data.scalars = valc

mlab.show()

In [38]:
# export solution data to file
dump.write({phi,mesh},filename='xxx')


In [28]:
# extract data from file
mesh, phi = dump.read('xxx')